In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import cv2
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
import torch.nn as nn
from torchvision import models, transforms, utils
from torchvision.transforms.functional import normalize, resize, to_pil_image
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
from torchcam.methods import GradCAM
from torchcam.utils import overlay_mask
from tqdm import tqdm

In [ ]:
def preprocess_image(image, size=(224, 224)):
    # Open and resize image
    # image = Image.open(image_path)
    image = image.resize(size, Image.Resampling.LANCZOS)
    
    # Convert the image to a numpy array and then to grayscale
    img = np.array(image)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Apply a binary threshold to get a binary image
    _, binary = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

   # Check if contours list is not empty
    if contours:
        # Find the largest contour which should be the fundus
        largest_contour = max(contours, key=cv2.contourArea)

        # Get the bounding rectangle for the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Crop the image to the bounding rectangle
        cropped = image.crop((x, y, x+w, y+h))
    else:
        print(f"No contours found in image")
        cropped = image  # or handle this case differently as per your requirement

    # Resize the cropped image to the desired size
    cropped = cropped.resize(size, Image.Resampling.LANCZOS)

    # # Color normalization
    # mean = np.mean(cropped, axis=(0, 1))
    # image_normalized = cropped - mean

    # Illumination correction and contrast enhancement using CLAHE
    image_lab = cv2.cvtColor(np.uint8(cropped), cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(image_lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    image_clahe = cv2.merge((l_clahe, a, b))
    image_clahe_rgb = cv2.cvtColor(image_clahe, cv2.COLOR_LAB2RGB)


    # Convert the image to uint8 before applying Gaussian blur
    image_clahe_rgb_uint8 = (image_clahe_rgb * 255).astype(np.uint8)

    # Apply Gaussian blur
    image_denoised = cv2.GaussianBlur(image_clahe_rgb_uint8, (5, 5), 0.5)

    image_denoised = Image.fromarray(image_denoised)
    return image_denoised

In [ ]:
def crop_fundus_image(image_path, save_path=None):
    # Load the image
    img = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply a binary threshold to get a binary image
    _, binary = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find the largest contour which should be the fundus
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Get the bounding rectangle for the largest contour
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Crop the image to the bounding rectangle
    cropped_img = img[y:y+h, x:x+w]
    
    # Save or return the cropped image
    if save_path:
        cv2.imwrite(save_path, cropped_img)
    else:
        return cropped_img

In [ ]:
def preprocess_all_images(dataset_dirs=['DDR-dataset/DR_grading/train', 'DDR-dataset/DR_grading/valid', 'DDR-dataset/DR_grading/test']):
    for dataset_dir in dataset_dirs:
        image_files = os.listdir(dataset_dir)
        for image_file in image_files:
            image_path = os.path.join(dataset_dir, image_file)
            image = Image.open(image_path)
            preprocessed_image = preprocess_image(image)
            save_path = os.path.join(dataset_dir + '_preprocessed', image_file)
            preprocessed_image.save(save_path)

# preprocess_all_images()

In [ ]:
train_df = pd.read_csv('DDR-dataset/DR_grading/train.txt', sep=' ', header=None, names=['image', 'label'])
train_df = train_df.query('label != 5')
train_df['path'] = train_df['image'].apply(lambda x: os.path.join('DDR-dataset/DR_grading/train_preprocessed', x))

In [ ]:
def balance_dataset(df):
    new_rows = []
    class_counts = np.bincount(df['label'])
    difference = np.max(class_counts) - class_counts
    aug_transforms = v2.Compose([
        v2.RandomHorizontalFlip(p=0.5),
        v2.RandomVerticalFlip(p=0.5),
        v2.RandomRotation(degrees=360),
        v2.RandomPerspective(distortion_scale=0, p=1, interpolation=3),
        v2.RandomAffine(degrees=0, translate=(0, 0), scale=(1, 1), shear=0),
        v2.ColorJitter(brightness=0.1, contrast=0.1, saturation=0, hue=0),
    ])
    for i, count in enumerate(difference):
        if count > 0:
            samples = df.query(f'label == {i}').sample(count, replace=True)
            # print(len(samples))
            for index, row in samples.iterrows():
                image = Image.open(row['path'])
                augmented_image = aug_transforms(image)
                augmented_image.save(f'{row["path"][:-4]}_augmented{count}.png')
                new_rows.append({'image': f'{row["image"][:-4]}_augmented{count}.png', 'label': i, 'path': f'{row["path"][:-4]}_augmented{count}.png'})
    new_df = pd.DataFrame(new_rows)
    new_df = pd.concat([df, new_df])
    return new_df

# new_df = balance_dataset(train_df)

# new_df.to_csv('DDR-dataset/DR_grading/train_augmented.txt', sep=' ', header=False, index=False)